In [2]:
from datasets import load_dataset

ds = load_dataset("chillies/vn-legal-conversation")

In [3]:
# Phân tích cấu trúc dataset
ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 31433
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 1747
    })
})

In [4]:
# Phân tích chi tiết dataset
import pandas as pd
import numpy as np

print("📊 Thông tin dataset:")
total_samples = len(ds['train']) + len(ds['validation']) + len(ds['test'])
print(f"- Train set: {len(ds['train'])} samples")
print(f"- Validation set: {len(ds['validation'])} samples") 
print(f"- Test set: {len(ds['test'])} samples")
print(f"- Total: {total_samples} samples")

print("\n📝 Các trường có sẵn:")
for feature in ds['train'].features:
    print(f"- {feature}")

print("\n🔍 Xem mẫu dữ liệu đầu tiên:")
sample = ds['train'][0]
for key, value in sample.items():
    if isinstance(value, str) and len(value) > 100:
        print(f"{key}: {value[:100]}...")
    else:
        print(f"{key}: {value}")

📊 Thông tin dataset:
- Train set: 31433 samples
- Validation set: 1746 samples
- Test set: 1747 samples
- Total: 34926 samples

📝 Các trường có sẵn:
- Unnamed: 0
- Question
- Answer

🔍 Xem mẫu dữ liệu đầu tiên:
Unnamed: 0: 2879
Question: Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay dượng tôi phản bội mẹ tôi...
Answer: Khoản 1 Điều 9 Luật hôn nhân và gia đình 2014 quy định: “Việc kết hôn phải được đăng ký và do cơ qua...


In [5]:
# Gộp tất cả các splits thành một dataset duy nhất để training
print("🔗 Gộp tất cả splits thành một dataset...")

# Combine all splits
all_data = []

# Add all train data
for item in ds['train']:
    all_data.append({
        'Question': item['Question'],
        'Answer': item['Answer']
    })

# Add all validation data
for item in ds['validation']:
    all_data.append({
        'Question': item['Question'], 
        'Answer': item['Answer']
    })

# Add all test data
for item in ds['test']:
    all_data.append({
        'Question': item['Question'],
        'Answer': item['Answer']
    })

print(f"✅ Đã gộp {len(all_data)} samples từ tất cả splits")

# Phân tích chất lượng dữ liệu
def analyze_text_quality(data):
    """Phân tích chất lượng text trong dataset"""
    questions = [item['Question'] for item in data]
    answers = [item['Answer'] for item in data]
    
    print(f"\n📈 Phân tích toàn bộ dataset:")
    
    # Độ dài câu hỏi
    question_lengths = [len(q) if q else 0 for q in questions]
    print(f"🔸 Độ dài câu hỏi:")
    print(f"   - Trung bình: {np.mean(question_lengths):.1f} ký tự")
    print(f"   - Min: {min(question_lengths)}, Max: {max(question_lengths)}")
    print(f"   - Median: {np.median(question_lengths):.1f}")
    
    # Độ dài câu trả lời
    answer_lengths = [len(a) if a else 0 for a in answers]
    print(f"🔸 Độ dài câu trả lời:")
    print(f"   - Trung bình: {np.mean(answer_lengths):.1f} ký tự")
    print(f"   - Min: {min(answer_lengths)}, Max: {max(answer_lengths)}")
    print(f"   - Median: {np.median(answer_lengths):.1f}")
    
    # Kiểm tra dữ liệu rỗng
    empty_questions = sum(1 for q in questions if not q or q.strip() == "")
    empty_answers = sum(1 for a in answers if not a or a.strip() == "")
    
    print(f"🔸 Dữ liệu rỗng:")
    print(f"   - Câu hỏi rỗng: {empty_questions}")
    print(f"   - Câu trả lời rỗng: {empty_answers}")
    
    return {
        'questions': questions,
        'answers': answers,
        'question_lengths': question_lengths,
        'answer_lengths': answer_lengths,
        'empty_questions': empty_questions,
        'empty_answers': empty_answers
    }

analysis = analyze_text_quality(all_data)

🔗 Gộp tất cả splits thành một dataset...
✅ Đã gộp 34926 samples từ tất cả splits

📈 Phân tích toàn bộ dataset:
🔸 Độ dài câu hỏi:
   - Trung bình: 327.0 ký tự
   - Min: 0, Max: 11938
   - Median: 217.0
🔸 Độ dài câu trả lời:
   - Trung bình: 1435.4 ký tự
   - Min: 47, Max: 26802
   - Median: 1177.0
🔸 Dữ liệu rỗng:
   - Câu hỏi rỗng: 3
   - Câu trả lời rỗng: 0


In [6]:
# Làm sạch và chuẩn hóa dữ liệu
def clean_text(text):
    """Làm sạch text"""
    if not text:
        return ""
    
    # Loại bỏ khoảng trắng thừa
    text = " ".join(text.split())
    
    # Loại bỏ ký tự đặc biệt không cần thiết
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    
    return text.strip()

def process_dataset(data, max_answer_length=8000):
    """Xử lý dataset và lọc dữ liệu chất lượng"""
    processed_data = []
    
    for item in data:
        question = clean_text(item['Question'])
        answer = clean_text(item['Answer'])
        
        # Lọc dữ liệu theo tiêu chí chất lượng
        if (len(question) >= 10 and  # Câu hỏi ít nhất 10 ký tự
            len(answer) >= 50 and    # Câu trả lời ít nhất 50 ký tự
            len(answer) <= max_answer_length and  # Giới hạn độ dài trả lời
            len(question) <= 2000):  # Giới hạn độ dài câu hỏi
            
            processed_data.append({
                'question': question,
                'answer': answer
            })
    
    return processed_data

print("🔧 Đang xử lý và làm sạch dữ liệu...")

# Xử lý toàn bộ dataset
processed_data = process_dataset(all_data)
print(f"✅ Dataset: {len(all_data)} → {len(processed_data)} (giữ lại {len(processed_data)/len(all_data)*100:.1f}%)")

print(f"\n📊 Tổng kết sau xử lý:")
print(f"- Tổng số mẫu training: {len(processed_data)}")
print(f"- Trung bình độ dài câu hỏi: {np.mean([len(item['question']) for item in processed_data]):.1f} ký tự")
print(f"- Trung bình độ dài câu trả lời: {np.mean([len(item['answer']) for item in processed_data]):.1f} ký tự")

🔧 Đang xử lý và làm sạch dữ liệu...
✅ Dataset: 34926 → 34566 (giữ lại 99.0%)

📊 Tổng kết sau xử lý:
- Tổng số mẫu training: 34566
- Trung bình độ dài câu hỏi: 306.9 ký tự
- Trung bình độ dài câu trả lời: 1409.2 ký tự
✅ Dataset: 34926 → 34566 (giữ lại 99.0%)

📊 Tổng kết sau xử lý:
- Tổng số mẫu training: 34566
- Trung bình độ dài câu hỏi: 306.9 ký tự
- Trung bình độ dài câu trả lời: 1409.2 ký tự


In [7]:
# Tạo thư mục và lưu dữ liệu vào data/finetune_data2
import os
import json

# Tạo thư mục data/finetune_data2 (khác với dataset trước)
output_dir = "../data/finetune_data3"
os.makedirs(output_dir, exist_ok=True)

print(f"📁 Tạo thư mục: {output_dir}")

# Lưu dữ liệu dưới nhiều format khác nhau

# 1. Format JSONL (mỗi dòng là một JSON object)
def save_jsonl(data, filepath):
    """Lưu dữ liệu dưới định dạng JSONL"""
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

# 2. Format instruction (cho fine-tuning)
def save_instruction_format(data, filepath):
    """Lưu dữ liệu dưới định dạng instruction tuning"""
    instruction_data = []
    for item in data:
        instruction_item = {
            "instruction": "Trả lời câu hỏi pháp luật sau:",
            "input": item['question'],
            "output": item['answer']
        }
        instruction_data.append(instruction_item)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in instruction_data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

# 3. Format conversation (cho chatbot training)
def save_conversation_format(data, filepath):
    """Lưu dữ liệu dưới định dạng conversation"""
    conversation_data = []
    for item in data:
        conversation_item = {
            "conversations": [
                {"role": "user", "content": item['question']},
                {"role": "assistant", "content": item['answer']}
            ]
        }
        conversation_data.append(conversation_item)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in conversation_data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

print("💾 Đang lưu dữ liệu...")

# Lưu QA format
qa_path = os.path.join(output_dir, "vilqa_qa_format.jsonl")
save_jsonl(processed_data, qa_path)
print(f"✅ Đã lưu QA format: {qa_path}")

# Lưu instruction format  
instruction_path = os.path.join(output_dir, "vilqa_instruction_format.jsonl")
save_instruction_format(processed_data, instruction_path)
print(f"✅ Đã lưu instruction format: {instruction_path}")

# Lưu conversation format
conversation_path = os.path.join(output_dir, "vilqa_conversation_format.jsonl")
save_conversation_format(processed_data, conversation_path)
print(f"✅ Đã lưu conversation format: {conversation_path}")

📁 Tạo thư mục: ../data/finetune_data3
💾 Đang lưu dữ liệu...
✅ Đã lưu QA format: ../data/finetune_data3\vilqa_qa_format.jsonl
✅ Đã lưu QA format: ../data/finetune_data3\vilqa_qa_format.jsonl
✅ Đã lưu instruction format: ../data/finetune_data3\vilqa_instruction_format.jsonl
✅ Đã lưu instruction format: ../data/finetune_data3\vilqa_instruction_format.jsonl
✅ Đã lưu conversation format: ../data/finetune_data3\vilqa_conversation_format.jsonl
✅ Đã lưu conversation format: ../data/finetune_data3\vilqa_conversation_format.jsonl


In [8]:
# Tạo metadata và tổng kết
metadata = {
    "dataset_info": {
        "source": "chillies/vn-legal-conversation",
        "description": "Vietnamese Legal Conversation Dataset processed for fine-tuning",
        "total_samples": len(processed_data),
        "original_splits": {
            "train": len(ds['train']),
            "validation": len(ds['validation']), 
            "test": len(ds['test'])
        },
        "combined_for_training": True
    },
    "processing_info": {
        "filters_applied": [
            "Minimum question length: 10 characters",
            "Maximum question length: 2000 characters",
            "Minimum answer length: 50 characters", 
            "Maximum answer length: 8000 characters",
            "Text cleaning: removed extra whitespace and special characters"
        ],
        "retention_rate": f"{len(processed_data)/len(all_data)*100:.1f}%"
    },
    "file_formats": {
        "qa_format": "Simple question-answer pairs",
        "instruction_format": "Instruction tuning format with instruction/input/output",
        "conversation_format": "Multi-turn conversation format for chatbot training"
    },
    "statistics": {
        "total_samples": len(processed_data),
        "avg_question_length": round(np.mean([len(item['question']) for item in processed_data]), 1),
        "avg_answer_length": round(np.mean([len(item['answer']) for item in processed_data]), 1)
    }
}

# Lưu metadata
metadata_path = os.path.join(output_dir, "vilqa_metadata.json")
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Đã lưu metadata: {metadata_path}")

# Hiển thị mẫu dữ liệu từ mỗi format
print("\n📝 Mẫu dữ liệu các format:")

print("\n🔸 QA Format:")
sample_qa = processed_data[0]
print(f"Question: {sample_qa['question'][:150]}...")
print(f"Answer: {sample_qa['answer'][:150]}...")

print("\n🔸 Instruction Format:")
print(f"Instruction: Trả lời câu hỏi pháp luật sau:")
print(f"Input: {sample_qa['question'][:150]}...")
print(f"Output: {sample_qa['answer'][:150]}...")

print("\n🔸 Conversation Format:")
print(f"User: {sample_qa['question'][:150]}...")
print(f"Assistant: {sample_qa['answer'][:150]}...")

print(f"\n🎉 Hoàn thành! Đã xử lý và lưu {len(processed_data)} mẫu dữ liệu vào {output_dir}")

# Hiển thị thống kê file
print(f"\n📁 Files đã tạo:")
for filename in os.listdir(output_dir):
    filepath = os.path.join(output_dir, filename)
    if os.path.isfile(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"   - {filename}: {size_mb:.2f} MB")

✅ Đã lưu metadata: ../data/finetune_data3\vilqa_metadata.json

📝 Mẫu dữ liệu các format:

🔸 QA Format:
Question: Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay dượng tôi phản bội mẹ tôi, có vợ mới và muốn chia đôi số tài sản, trong đó ...
Answer: Khoản 1 Điều 9 Luật hôn nhân và gia đình 2014 quy định: “Việc kết hôn phải được đăng ký và do cơ quan nhà nước có thẩm quyền thực hiện theo quy định c...

🔸 Instruction Format:
Instruction: Trả lời câu hỏi pháp luật sau:
Input: Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay dượng tôi phản bội mẹ tôi, có vợ mới và muốn chia đôi số tài sản, trong đó ...
Output: Khoản 1 Điều 9 Luật hôn nhân và gia đình 2014 quy định: “Việc kết hôn phải được đăng ký và do cơ quan nhà nước có thẩm quyền thực hiện theo quy định c...

🔸 Conversation Format:
User: Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay dượng tôi phản bội mẹ tôi, có vợ mới và muốn chia đôi số tài sản, trong đó .